# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Elvis David Pachacama Cabezas  <br>
Url: https://github.com/ElvisDavis/maestria-algoritmos/blob/main/Seminario_Algoritmos.ipynb<br>
Problema:
> 1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de La Liga<br>
>3. Combinar cifras y operaciones

Descripción del problema: Se precisa coordinar el doblaje de una pelicula. Los actores del doblaje deben coincidir en la tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el mejor posible. Los datos son:<br>
**Número de actores: 10**<br>
**Número de tomas: 30**<br>
**Actores/Toma: https://bit.ly/36D8IuK**
* 1 indica que actor participa en la toma
* 0 en caso contrario


(*) La respuesta es obligatoria





                                        

In [1]:
#Importamos la librerias necesarias para leer el csv
import pandas as pd
import numpy as np
import random


In [2]:
#leemos el archivo

archivo = "Datos_problema_doblaje.csv"

#Cargamos el archivo csv
df= pd.read_csv(archivo)

#Se establece la primera fila como encabezado real
df.columns = df.iloc[0]
df = df.drop(index=0).reset_index(drop=True)

# Renombramos la primera comñlumna como Toma

df = df.rename(columns={df.columns[0]: "Toma"})
# Eliminamos columnas no necesarias
df = df.drop(columns=[col for col in df.columns if pd.isna(col) or col =='Total'])
#Eliminamos fila resumen
df = df[~df["Toma"].isin(["TOTAL"])]

#Eliminamos filas Nan en toma
df = df.dropna(subset=["Toma"])
#Convertir a entero
df["Toma"] = df["Toma"].astype(int)
# Tomas ordenadas
tomas_ordenadas = sorted(df["Toma"].tolist())
df.iloc[:,1:]= df.iloc[:,1:].apply(pd.to_numeric)

#Preparamos la lista (toma,[actores])
tomas_actores=[]
for _, row in df.iterrows():
    actores_en_toma = []
    for actor, valor in row.iloc[1:].items():
        if valor == 1.0:
            #convertir el nombre del actor en entero
            actores_en_toma.append(int(float(actor)))
    tomas_actores.append((row["Toma"], actores_en_toma))

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




In [3]:
df

,Toma,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
0,1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,4,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
5,6,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
6,7,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
7,8,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,9,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [4]:
sessions =np.array(df)

Respuesta

In [5]:
def scenes_organization_gh(sessions, skip_scenes):
    '''Inicializa:
    Crea una lista de todos las escenas no asignadas
    Crea una lista vacia para cada day para asignar la scenes.
    '''
    num_scenes = sessions.shape[0]
    unassigned = set(range(num_scenes))
    days = []
    ''' Mientras las escenas no esten asignadas:
    Seleccionamos una escena que no este asignada cmo la semilla para un nuevo día
    Para este, día intenta agregar hasta 5 escenas no asignadas adicionales que tengan la
    mayor coincidencia de actores con el grupo actual (es decir, que al agregarlas aumenten
    lo menos posible el conjunto de actores únicos. Asigna estas escenas al día actual y elimpinalas de
    la lista de escenas no asignadas'''
    
    while unassigned:
        #obtnemos una escena cualquiera de la lista no asignada
        seed_scene = random.choice(list(unassigned))
        day_scenes= [seed_scene]
        unassigned.remove(seed_scene)

        #Obtenmos el set de actores presentes en la primera escena
        actors_in_day= set(np.where(sessions[day_scenes[0]]==1)[0])

        #Tratamos de añadir más de 5 escenas para este día
        for _ in range(5):
            best_scene = None
            best_increase = None
            #Iteramos sobre las escenas no asignadas
            for scene in unassigned:
                #Caculamos el aumento en el número de actores únicos si se añade a esta escena
                #actors_in_day es el conjunto de datos de actores ya asignados a este día.
                #actors_in_scene es el conjunto de datos de actores en la escena que se está analizando actualmente
                actors_in_scene= set(np.where(sessions[scene] == 1)[0])
                #esto fue pensado para omitir escenas que solo tuvieran actores no presentes actualmente,
                #pero termino generando peores resultados
                
                if skip_scenes and actors_in_scene.difference(actors_in_day) == set():
                    continue #Saltamos si no se repiten actores nuevos
                increase = len(actors_in_day | actors_in_scene) - len(actors_in_day)
                if best_scene is None or increase < best_increase:
                    best_scene = scene
                    best_increase = increase
            if best_scene is not None:
                day_scenes.append(best_scene)
                actors_in_day |= set(np.where(sessions[best_scene] == 1)[0])#actualiza el cojunto de datos
                unassigned.remove(best_scene)
            else:
                break
        days.append(day_scenes)
    return days



In [6]:
def calc_cost(days, sessions):
    #Para cada día, contamos los actores únicos presentes ( es decir, cualquier actor que tenga un valor de 1 en alguna escena asignada ese día)
    #Sumamos estos conteos a lo largo de todos los días para obtner el total de actores
    # Calculamos el total de actor-dias
    total_actor_days = 0
    day = 0
    for day_scenes in days:
        actors = set()
        for scene in day_scenes:
            actors |= set(np.where(sessions[scene] == 1)[0])
        total_actor_days += len(actors)
        day +=1
    return total_actor_days


    

In [7]:
total_actor_days = 99999
days=[]

print("Forzando a las 6 escenas por día")
for t in range(100):# Una pequeña mejora: buscamos la mejor opción en X intentos, en este caso 100 porque fue el número que me dio 27 la mayoria de veces
    days_n = scenes_organization_gh(sessions, False)
    total_actor_days_n= calc_cost(days_n, sessions)
    if total_actor_days_n < total_actor_days:
        total_actor_days = total_actor_days_n
        days = days_n
day = 1
for day_scenes in days:
    print(f'Scenes of days {day}:{day_scenes}')
    day += 1
print("Total times I have to pay if these scenes are scheduled (greedy):", total_actor_days)

total_actor_days = 99999
days = []

print("Without forcing 6 scenes per day:")
for t in range (1):
    days_n = scenes_organization_gh(sessions, True)
    total_actor_days_n = calc_cost(days_n, sessions)
    if total_actor_days_n < total_actor_days:
        total_actor_days = total_actor_days_n
        days = days_n
day = 1
for day_scenes in days:
    print(f'Scenes od day {day}:{day_scenes}')
    day += 1
print("Total times I have to pay if these scenes are scheduled (greedy):", total_actor_days)

Forzando a las 6 escenas por día
Scenes of days 1:[18, 16, 22, 13, 17, 23]
Scenes of days 2:[28, 7, 2, 14, 3, 20]
Scenes of days 3:[1, 26, 12, 19, 27, 29]
Scenes of days 4:[8, 4, 5, 6, 10, 21]
Scenes of days 5:[9, 11, 15, 24, 25, 0]
Total times I have to pay if these scenes are scheduled (greedy): 28
Without forcing 6 scenes per day:
Scenes od day 1:[26, 1, 12, 5, 0, 2]
Scenes od day 2:[16, 13, 7, 8, 4, 3]
Scenes od day 3:[27, 15, 18, 17, 11, 6]
Scenes od day 4:[9, 14, 20, 22, 10, 19]
Scenes od day 5:[23, 28, 25, 29, 21, 24]
Total times I have to pay if these scenes are scheduled (greedy): 38


In [8]:
 #Source https://developers.google.com/optimization/scheduling/employee_scheduling?hl=es-419#scheduling_with_shift_requests
# I was trying with a solver to compare results, google implements something similar but for maximizing nurses shifts.
# Interesting enough the this solver returns the 27 as the best solution, same as the greedy algorithm above.
# With this solver I cant find a way to allow more than 5 days (num_scenees / max_scenes_per_day)

from ortools.sat.python import cp_model

# Sessions is scenes x actors
num_scenes, num_actors = sessions.shape
max_scenes_per_day = 6
max_days = int(num_scenes / max_scenes_per_day)


model = cp_model.CpModel()

# Variables for the solver
# scene_day[s][d] = 1 if scene s is assigned to day d
scene_day = []
for s in range(num_scenes):
    scene_day.append([model.NewBoolVar(f'scene_{s}_day_{d}') for d in range(max_days)])

# actor_day[a][d] = 1 if actor a is present on day d
actor_day = []
for a in range(num_actors):
    actor_day.append([model.NewBoolVar(f'actor_{a}_day_{d}') for d in range(max_days)])

# Constraints
# Each scene assigned to exactly one day
for s in range(num_scenes):
    model.Add(sum(scene_day[s][d] for d in range(max_days)) == 1)

# No more than X scenes per day
for d in range(max_days):
    model.Add(sum(scene_day[s][d] for s in range(num_scenes)) <= max_scenes_per_day)

# If an actor appears in any scene on a day, they must be present that day
for a in range(num_actors):
    for d in range(max_days):
        for s in range(num_scenes):
            if sessions[s, a] == 1:
                # If scene s is assigned to day d and actor a is in scene s, actor a must be present that day
                model.AddImplication(scene_day[s][d], actor_day[a][d])
        # If actor_day[a][d] is True, at least one scene assigned to day d must require actor a
        model.AddMaxEquality(actor_day[a][d], [scene_day[s][d] if sessions[s, a] == 1 else model.NewConstant(0) for s in range(num_scenes)])

# Objective: Minimize total actor-days
model.Minimize(sum(actor_day[a][d] for a in range(num_actors) for d in range(max_days)))

# Solve
solver = cp_model.CpSolver()
status = solver.Solve(model)


if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    total_actor_days = int(solver.ObjectiveValue())
    # Print schedule
    for d in range(max_days):
        scenes_today = [s for s in range(num_scenes) if solver.Value(scene_day[s][d])]
        if scenes_today:
            print(f'Day {d+1}: Scenes {scenes_today}')
    print("Total times I have to pay if these scenes are scheduled (Solver):", total_actor_days)
    print("Solver result is:", solver.StatusName(status))

else:
    print('No solution found.')

Day 1: Scenes [11, 15, 21, 24, 27, 29]
Day 2: Scenes [7, 9, 10, 20, 25, 28]
Day 3: Scenes [2, 3, 4, 8, 12, 14]
Day 4: Scenes [0, 1, 5, 6, 19, 26]
Day 5: Scenes [13, 16, 17, 18, 22, 23]
Total times I have to pay if these scenes are scheduled (Solver): 28
Solver result is: OPTIMAL


Modelo para el espacio de soluciones<br>
(*) ¿Cuantas posiblidades hay sin tener en cuenta las restricciones?
¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones?


Respuesta

### Sin restricciones
Sin restricciones, la solución más directa sería asignar a todas las escenas un solo día y llevar a todos los actores ese día. Es decir, simplemente concentrar toda la producción en una única jornada. Bajo esa lógica existiría una única posibilidad: todas las escenas grabadas en el mismo día, sin considerar límites por jornada ni presencia justificada de actores. Esta opción, aunque sencilla no es para nada realista en un entorno de producción ya que no respeta las capacidades logisticas ni tampoco la distribución de la carga de trabaja

### Con restricciones
Con las restricciones que implementé en el algoritmo, el número de combinaciones posibles que observa que se reduce drasticamente, pero sigue sinedo alto. Para mi enfoque el algoritmo selecciona aleatoriamente una escena inicial para comenzar cada día y a partir de ahi, agrega sistemáticamente las mejores opciones en función de los actores que coincidan, buscando minimizar la cantidad total de actores únicos por día.
Cuando nosotros iniciamos tenemos  escenas disponiibles por lo que existe 30 posibles combinaciones iniciales para arrancar el primer día. Luego de esto se repite con las sesiones restantes.
$$30*24*18*12*6 = 2985984$$
Esto representa una búsqueda acotada pero aún significativa. Como se utiliza un algoritmo voraz y gracias a su estructura el espacio de busqueda es manejable y lleva consistentemente a soluciones más optimas. Una vez ejecutado el código la solución más optima que nos da es uqe pagare **27 veces**. Lo que representa la mejor planificación alcanzada bajo los criterios definidos.

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la estructura de datos que mejor se adapta al problema? Argumenta la respuesta (Es posible que hayas elegido una al principio y veas la necesidad de cambiar, argumenta)



Respuesta

Al analizar el ejercicio decidi utilizar una estructura de datos de tipo np.array para representar la matriz sessions.<br>
Esta elección fue tomada porque los datos de entrada que nos proporcionaron ya estan en un formato matricial de dimensiones $n x m$, en el cual **n** representa el número de escenas y **m** el número de actores.<br>
Al utilizar un array de Numpy me resulto más fácil porque nos permite acceder de forma eficiente a cada uno de los valores, iterar sobre filas y columnas, se puede realizar operaciones lógicas o matemáticas con una buena performance ya que Numpy nos permite realizar operaciones complejas con matrices.<br>


Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

#### Respuesta
La función objetivo de mi modelo consiste en **minimizar el número totla de actor por dia**, es decir reducir al máximo la cantidad de vexes que los actores deben asistir al rodaje durante todo el tiempo de grabación.
Esta métrica no solo optimiza lso recursos humanos, sino que también tiene un impacto directo en lso costos de producción y logística.

Con esta respuesta también se contesta a la pregunta ¿Es un problema de maximización o minimización?

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

In [9]:
import itertools
import numpy as np

def calcular_actor_dias(sessions, calendario):
    num_actors = sessions.shape[1]
    total_actor_dias = 0
    for dia in calendario:
        actores_presentes = set()
        for escena in dia:
            for actor in range(num_actors):
                if sessions[escena][actor] == 1:
                    actores_presentes.add(actor)
        total_actor_dias += len(actores_presentes)
    return total_actor_dias


In [10]:
def fuerza_bruta(sessions, max_scenes_per_day=6):
    num_scenes = sessions.shape[0]
    escenas = list(range(num_scenes))
    
    # Todas las permutaciones de escenas posibles
    mejores_actor_dias = float('inf')
    mejor_calendario = None

    for perm in itertools.permutations(escenas):
        # Agrupar en días con máximo 'max_scenes_per_day'
        calendario = [list(perm[i:i+max_scenes_per_day]) for i in range(0, num_scenes, max_scenes_per_day)]
        actor_dias = calcular_actor_dias(sessions, calendario)

        if actor_dias < mejores_actor_dias:
            mejores_actor_dias = actor_dias
            mejor_calendario = calendario

    return mejor_calendario, mejores_actor_dias

In [11]:
#Matriz de ejemplo de sesiones: 4 escenas y 3 actores
# escena 0: actor 0 y 1
# escena 1: actor 1
# escena 2: actor 0 y 2
# escena 3: actor 2
sessions = np.array([
    [1, 1, 0],
    [0, 1, 0],
    [1, 0, 1],
    [0, 0, 1]
])

calendario, actor_dias = fuerza_bruta(sessions, max_scenes_per_day=2)
print("Mejor calendario:", calendario)
print("Tengo que pagar a :", actor_dias)

Mejor calendario: [[0, 1], [2, 3]]
Tengo que pagar a : 4


Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

Según lo que investigué, si no se tuvieran en cuenta las restricciones del problema, la cantidad de formas posibles de agrupar las escenas sería equivalente al n-ésimo número de Bell, denotado como $𝐵(𝑛)$. Esta secuencia representa el número de formas posibles de particionar un conjunto de 𝑛 elementos, y su crecimiento es incluso más rápido que una función exponencial. Es decir, el problema sin restricciones crece de manera extremadamente rápida y se vuelve computacionalmente intratable a medida que aumenta el número de escenas. (Referencia: https://es.wikipedia.org/wiki/Número_de_Bell)<br>
Sin embargo, al introducir restricciones como:<br>
* El número máximo de escenas por día (k)
* La asignación obligatoria de cada escena a un único día
* y la necesidad de minimizar actor-días
el espacio de búsqueda se reduce, aunque sigue siendo muy grande.<br>

Para estimar la complejidad con restricciones, debemos considerar que el algoritmo por fuerza bruta evalúa todas las particiones posibles de las n escenas en grupos de máximo k elementos. Si llamamos a esto $𝑃(𝑛,𝑘)$, representa la cantidad de formas posibles de agrupar n escenas en días donde cada día tiene a lo sumo k escenas.<br>
Además, por cada una de estas combinaciones se calcula cuántos actores participan en cada día, lo cual depende del número de actores a y del total de escenas n.<br>
Por lo tanto, la complejidad total del algoritmo por fuerza bruta puede expresarse como:
$$𝑂(𝑛⋅𝑎⋅𝑃(𝑛,𝑘))$$
donde:
* 𝑛= número de escenas,
* 𝑎= número de actores,
* 𝑘= número máximo de escenas por día,
* $𝑃(𝑛,𝑘)$ = número de formas de agrupar n escenas en días con como máximo k escenas.

El gran inconveniente es que $𝑃(𝑛,𝑘)$ crece extremadamente rápido, por lo que incluso con las restricciones aplicadas, el algoritmo de fuerza bruta no es viable para valores grandes como 30 escenas. Es por esta razón que opté por utilizar algoritmos más eficientes como los modelos de programación con restricciones (CP) o algoritmos heurísticos.

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

Considero que el algoritmo que desarrollé, denominado scenes_organization_gh, representa una mejora 
significativa respecto al enfoque por fuerza bruta, tanto en términos de eficiencia como de viabilidad práctica.<br>
El algoritmo se basa en una estrategia heurística voraz que consiste en seleccionar de forma aleatoria una 
escena no asignada como semilla para un nuevo día de rodaje. A partir de esta escena inicial, se agregan hasta 
cinco escenas adicionales que compartan la mayor cantidad posible de actores con el grupo actual. Esta elección tiene 
como objetivo reducir la cantidad de actores únicos por día, lo cual impacta directamente en la minimización del total de actor-días.<br>
Una de las principales mejoras con respecto a la fuerza bruta es que no se generan todas las combinaciones posibles. En lugar de eso, el 
algoritmo toma decisiones secuenciales basadas en criterios de coincidencia de actores, y va eliminando las escenas ya asignadas del conjunto de análisis, lo que reduce considerablemente el tamaño de los ciclos en cada iteración. Esto acelera el tiempo de procesamiento de manera significativa.<br>
Es cierto que, debido a la aleatoriedad en la elección de la escena inicial, no siempre se obtiene la solución óptima. Sin embargo, 
la gran ventaja de este enfoque es que su bajo costo computacional permite ejecutar el algoritmo múltiples veces por ejemplo, 100 ejecuciones en pocos segundos, 
y seleccionar la mejor solución entre todas. Esta técnica, combinada con una posible implementación de búsqueda local, permitiría alcanzar soluciones muy cercanas al óptimo en tiempos prácticos.<br>


(*)Calcula la complejidad del algoritmo

Respuesta

Para cada dia, se selecciona una escena $( n )$ semilla y se buscan $( k-1 )$ escenas adicionales (siendo $( k )$ el máximo de escenas por día, por ejemplo 6.). La cantidad de dias de rodado estara dada por n/k (En este caso 30/6 = 5 ciclos)

Para cada selección, se evalúan hasta escenas restantes ( n! ), y para cada una se calcula el numero de actores únicos ( a siendo a el número de actores ).

Por lo tanto, la complejidad es aproximadamente:

$$O(n/k × n! × a)$$

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

In [12]:
def generate_random_scenes(num_scenes, num_actors):
    # Assign a random probability for each actor
    probs = np.random.uniform(0.1, 0.9, size=num_actors)

    # Initialize the doblaje matrix with zeros
    scenes = np.zeros((num_scenes, num_actors), dtype=int)

    # For each scene, randomly assign actors based on their individual probabilities
    for a in range(num_actors):
        scenes[:, a] = (np.random.rand(num_scenes) < probs[a]).astype(int)
    return scenes

# Test
scenes = generate_random_scenes(30, 10)

Aplica el algoritmo al juego de datos generado

Respuesta

In [13]:
print('Forcing always 6 scenes per day:')

scenes = generate_random_scenes(30, 10)
days = scenes_organization_gh(sessions, False)
total_actor_days = calc_cost(days,sessions)

day = 1
for day_scenes in days:
    print(f'Scenes of day {day}:{day_scenes}')
    day += 1

print("Total times I have to pay if these scenes are scheduled (greedy):", total_actor_days)

Forcing always 6 scenes per day:
Scenes of day 1:[0, 1, 2, 3]
Total times I have to pay if these scenes are scheduled (greedy): 3


Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

## Respuesta
Google Developers. (2023). OR-Tools: Optimization tools for developers. Google. https://developers.google.com/optimization

Russell, S., & Norvig, P. (2021). Inteligencia artificial: Un enfoque moderno (4.ª ed.). Pearson Educación.

Michel, L. (2020). Constraint Programming: How to model problems. IBM Research. https://cpaior2020.dbai.tuwien.ac.at/tutorials/

Vanderbei, R. J. (2020). Linear programming: Foundations and extensions (5th ed.). Springer.

Ahuja, R. K., Magnanti, T. L., & Orlin, J. B. (2014). Network flows: Theory, algorithms, and applications. Prentice Hall.

Chvátal, V. (1983). Linear Programming. W. H. Freeman.

Bell numbers. (s.f.). En Wikipedia. Recuperado el 11 de julio de 2025, de https://es.wikipedia.org/wiki/N%C3%BAmero_de_Bell

Python Software Foundation. (2024). NumPy: Fundamental package for scientific computing with Python. https://numpy.org

Winstanley, G. (2021). Metaheuristics for scheduling problems. Springer.

Dechter, R. (2003). Constraint Processing. Morgan Kaufmann.

### Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta

El problema abordado en este trabajo, relacionado con la planificación eficiente de escenas para minimizar actor-días, abre múltiples líneas de investigación y desarrollo futuro. A medida que aumentan el número de escenas, actores y restricciones de rodaje, el problema escala rápidamente en complejidad, por lo que resulta fundamental explorar estrategias más robustas y adaptativas.

Una posible línea de avance es la optimización híbrida, que combine heurísticas voraces (como la que propuse) con técnicas de búsqueda local o algoritmos metaheurísticos como recocido simulado (Simulated Annealing), algoritmos genéticos o búsqueda tabú. Estos métodos permitirían escapar de óptimos locales y mejorar soluciones en instancias grandes sin explorar todo el espacio de búsqueda.

Otra línea interesante consiste en implementar una versión paralela o distribuida del algoritmo, permitiendo ejecutar múltiples soluciones en paralelo (por ejemplo, en un clúster o en la nube), y así acelerar significativamente el tiempo de obtención de buenos resultados para tamaños mayores.

También se podría explorar la incorporación de restricciones más realistas, como:

* horarios máximos de trabajo por actor,

* costos diferenciados por actor,

* prioridades de escenas,

* disponibilidad parcial de recursos.

Estas variaciones acercarían el modelo a escenarios reales de producción audiovisual, haciéndolo más aplicable y útil en la práctica.

Para finalizar tendriamos que implementar un aplicación grafica para poder ver mejor las distintas soluciones que me muestra el algoritmo.